In [1]:
import os

# 1. 추가하려는 경로를 정의합니다.
# Windows 경로를 다룰 때는 raw string(r"...")을 사용하는 것이 좋습니다.
ffmpeg_path = r"D:\Study\GPT_AGENT_2025_BOOK\ffmpeg-7.1.1-full_build\bin"

# 2. 현재 PATH 환경 변수를 가져옵니다.
# os.pathsep은 운영체제에 맞는 경로 구분자(';' 또는 ':')를 자동으로 사용해줍니다.
path_list = os.environ.get('PATH', '').split(os.pathsep)

# 3. 추가하려는 경로가 이미 PATH에 있는지 확인합니다.
if ffmpeg_path not in path_list:
    # 4. PATH에 없는 경우에만 맨 앞에 추가합니다.
    os.environ['PATH'] = ffmpeg_path + os.pathsep + os.environ.get('PATH', '')
    print(f"'{ffmpeg_path}' 경로를 PATH에 추가했습니다.")
else:
    print(f"'{ffmpeg_path}' 경로는 이미 PATH에 존재합니다.")

# (선택 사항) 변경된 PATH 확인
print("\n--- 현재 PATH ---")
print(os.environ['PATH'])


'D:\Study\GPT_AGENT_2025_BOOK\ffmpeg-7.1.1-full_build\bin' 경로는 이미 PATH에 존재합니다.

--- 현재 PATH ---
d:\Study\GPT_AGENT_2025_BOOK\venv\Scripts;D:\Study\GPT_AGENT_2025_BOOK\venv\Scripts;C:\Program Files\Eclipse Foundation\jdk-8.0.302.8-hotspot\bin;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\ProgramData\Oracle\Java\javapath;C:\Program Files (x86)\M1 Licensing;c:\bd5\Delphi5\Projects\Bpl;c:\bd5\vbroker\jre\Bin;c:\bd5\vbroker\Bin;c:\bd5\Delphi5\Bin;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files (x86)\Common Files\Adobe\AGL;C:\Program Files (x86)\Microsoft SQL Server\80\Tools\BINN;C:\Program Files (x86)\GE\iFIX;C:\Program Files\dotnet\;C:\Program Files\Microsoft SQL Server\130\Tools\Binn\;C:\Program Files\Microsoft SQL Server\Client SDK\ODBC\170\Tools\Binn\;C:\Pro-face\WinGP\SDK\;C:\Program Files (x86)\Microsoft SQL Server\140\Tools

In [2]:
import os
import sys
import winreg
import ctypes

# ==============================================================================
# 중요: 1. 아래 FFMPEG_BIN_PATH에 본인의 FFmpeg bin 폴더 경로를 정확히 입력하세요.
#         예: "C:\\ffmpeg\\bin" (백슬래시를 두 번 사용하거나 슬래시를 사용하세요)
#       2. 이 셀을 실행하려면 Jupyter/VSCode를 "관리자 권한으로 실행"해야 할 수 있습니다.
# ==============================================================================
FFMPEG_BIN_PATH = r"D:\Study\GPT_AGENT_2025_BOOK\ffmpeg-7.1.1-full_build\bin" # <--- ‼️ 본인의 FFmpeg bin 폴더 경로로 수정하세요.

def add_path_to_system_environment(path_to_add):
    """
    Windows 시스템 환경 변수 PATH에 새로운 경로를 영구적으로 추가합니다.
    관리자 권한이 필요하며, 변경 사항을 적용하려면 재시작이 필요합니다.
    """
    if not os.path.isdir(path_to_add):
        print(f"오류: '{path_to_add}'는 유효한 디렉터리가 아닙니다. 경로를 확인해주세요.")
        return

    try:
        # 시스템 환경 변수에 접근하기 위해 레지스트리 키를 엽니다.
        # HKEY_LOCAL_MACHINE은 시스템 전체에, HKEY_CURRENT_USER는 현재 사용자에게만 적용됩니다.
        # 시스템 전체에 적용하려면 관리자 권한이 필요합니다.
        key = winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, 
                             r'SYSTEM\CurrentControlSet\Control\Session Manager\Environment', 
                             0, 
                             winreg.KEY_READ | winreg.KEY_WRITE)

        # 현재 PATH 값을 읽어옵니다.
        current_path_value, reg_type = winreg.QueryValueEx(key, 'Path')

        # 경로들을 세미콜론(;)으로 분리합니다.
        paths = current_path_value.split(';')

        # 추가하려는 경로가 이미 포함되어 있는지 확인합니다.
        if path_to_add in paths or path_to_add + '\\' in paths:
            print(f"'{path_to_add}' 경로는 이미 시스템 PATH에 존재합니다.")
        else:
            # 새로운 경로를 추가합니다.
            new_path_value = current_path_value + ';' + path_to_add
            winreg.SetValueEx(key, 'Path', 0, reg_type, new_path_value)
            print(f"'{path_to_add}' 경로를 시스템 PATH에 성공적으로 추가했습니다.")
            
            # 모든 프로그램에 변경 사항을 알립니다.
            HWND_BROADCAST = 0xFFFF
            WM_SETTINGCHANGE = 0x1A
            ctypes.windll.user32.SendMessageW(HWND_BROADCAST, WM_SETTINGCHANGE, 0, "Environment")
            print("모든 프로그램에 환경 변수 변경을 알렸습니다.")

        winreg.CloseKey(key)

    except PermissionError:
        print("\n[오류] 권한이 거부되었습니다.")
        print("이 스크립트를 시스템 PATH에 쓰려면 관리자 권한이 필요합니다.")
        print("Jupyter Notebook 또는 VS Code를 '관리자 권한으로 실행'한 후 다시 시도해주세요.")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")

# 함수 실행
add_path_to_system_environment(FFMPEG_BIN_PATH)

print("\n======================================================================")
print("‼️ 중요: 변경 사항을 적용하려면 반드시 커널을 재시작하거나")
print("   VS Code/터미널을 완전히 종료 후 다시 실행해주세요!")
print("======================================================================")



'D:\Study\GPT_AGENT_2025_BOOK\ffmpeg-7.1.1-full_build\bin' 경로는 이미 시스템 PATH에 존재합니다.

‼️ 중요: 변경 사항을 적용하려면 반드시 커널을 재시작하거나
   VS Code/터미널을 완전히 종료 후 다시 실행해주세요!


In [8]:
# instantiate the pipeline with safer token handling and diagnostics
from pyannote.audio import Pipeline
from dotenv import load_dotenv
import os, traceback
import torch
import soundfile as sf
import numpy as np

print("cwd:", os.getcwd())
# load .env (if present). load_dotenv returns True if a file was found
ok = load_dotenv()
print("load_dotenv returned:", ok)

# Prefer explicit token passing. Do NOT print the token value here for security.
token = os.getenv("HF_TOKEN")
if token:
    print("HF token found in environment (value not shown).")
else:
    print("No HF_TOKEN in environment. If the model is gated, you must set HF_TOKEN with a token that has Read permission.")

# Try to instantiate the pyannote pipeline. If it fails, capture and print guidance.
try:
    if token:
        pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", token=token)
    else:
        pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1")
    print("Pipeline loaded successfully.")
except Exception as e:
    print("Failed to load Pipeline.from_pretrained()")
    traceback.print_exc()
    print("If the model is gated: create an HF token with Read permission and either set it as HF_TOKEN in your environment or pass it with token=token.")
    print("You can also accept the model terms on the Hugging Face model page if required.")
    raise

# GPU: try to move pipeline to GPU but handle failures gracefully
try:
    if torch.cuda.is_available():
        try:
            pipeline.to(torch.device("cuda:0"))
            print("CUDA available — pipeline moved to GPU.")
        except Exception as e:
            print("Could not move pipeline to GPU (possibly OOM). Continuing on CPU.")
            traceback.print_exc()
    else:
        print("CUDA is not available — using CPU.")
except NameError:
    # pipeline variable might not exist if instantiation failed
    print("Pipeline not available to move to GPU. Skipping GPU move.")


cwd: d:\Study\GPT_AGENT_2025_BOOK\chap05\sec03
load_dotenv returned: True
HF token found in environment (value not shown).
Pipeline loaded successfully.
CUDA available — pipeline moved to GPU.


In [9]:
# ==============================================================================
# [Workaround] torchcodec/torchaudio compatibility fix
# ==============================================================================
# Since torchcodec is incompatible with the current PyTorch version, we use
# soundfile to load audio and pass the waveform directly to the pipeline.

def load_audio(file_path):
    """
    Load audio file using soundfile and convert to PyTorch tensor.
    Returns a dictionary suitable for pyannote.audio pipeline.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Audio file not found: {file_path}")
        
    # Load audio using soundfile (returns numpy array)
    # data shape: (time, channels) or (time,)
    data, sr = sf.read(file_path)
    
    # Convert to PyTorch tensor
    waveform = torch.from_numpy(data).float()
    
    # Ensure shape is (channels, time)
    if waveform.ndim == 1:
        waveform = waveform.unsqueeze(0)
    else:
        waveform = waveform.t()
        
    return {"waveform": waveform, "sample_rate": sr}

print("Workaround function 'load_audio' defined. Use this to load audio files.")

# Example Usage:
# audio_file = "path/to/your/audio.wav"
# io = load_audio(audio_file)
# diarization = pipeline(io)


Workaround function 'load_audio' defined. Use this to load audio files.


In [11]:
io = load_audio("../audio/싼기타_비싼기타.mp3")
diarization = pipeline(io)

with open("../audio/싼기타_비싼기타.rttm", "w", encoding="utf-8") as rttm:
    diarization.speaker_diarization.write_rttm(rttm)

#print(diarization)

In [14]:
import pandas as pd
rttm_path = "../audio/싼기타_비싼기타.rttm"
df_rttm = pd.read_csv(
    rttm_path,
    sep =" ",
    header=None,
    names=["type", "file", "chnl", "start", "duration", "C1", "C2", "speaker_id", "C3", "C4"]
    )
print(df_rttm)

       type      file  chnl    start  duration  C1  C2  speaker_id  C3  C4
0   SPEAKER  waveform     1    0.993     5.805 NaN NaN  SPEAKER_00 NaN NaN
1   SPEAKER  waveform     1    7.405     3.983 NaN NaN  SPEAKER_00 NaN NaN
2   SPEAKER  waveform     1   11.759     4.927 NaN NaN  SPEAKER_00 NaN NaN
3   SPEAKER  waveform     1   17.210    10.665 NaN NaN  SPEAKER_00 NaN NaN
4   SPEAKER  waveform     1   28.668     1.536 NaN NaN  SPEAKER_00 NaN NaN
..      ...       ...   ...      ...       ...  ..  ..         ...  ..  ..
83  SPEAKER  waveform     1  414.481     2.970 NaN NaN  SPEAKER_01 NaN NaN
84  SPEAKER  waveform     1  417.755     3.476 NaN NaN  SPEAKER_00 NaN NaN
85  SPEAKER  waveform     1  423.644     0.776 NaN NaN  SPEAKER_01 NaN NaN
86  SPEAKER  waveform     1  424.741     3.527 NaN NaN  SPEAKER_01 NaN NaN
87  SPEAKER  waveform     1  428.504     0.844 NaN NaN  SPEAKER_01 NaN NaN

[88 rows x 10 columns]


In [ ]:
# start + duration을 end로 변환
df_rttm['end'] = df_rttm['start'] + df_rttm['duration']
display(df_rttm)

0       6.798
1      11.388
2      16.686
3      27.875
4      30.204
       ...   
83    417.451
84    421.231
85    424.420
86    428.268
87    429.348
Length: 88, dtype: float64